In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# MuxComp and DemuxComp

```{Note}
DemuxComp is being deprecated in favor of `om.slicer` for a safer and more robust solution. [Below](om_slicer) is an example of how to use `om.slicer` to achieve the same result.
```

`DemuxComp` and `MuxComp` work together to break up inputs into multiple values (demux) or combine
multiple inputs into a single value (mux).  This can be useful in situations where scalar outputs
from multiple components need to be fed into a single vectorized component.

`DemuxComp` takes a single input of arbitary shape (the size of at least one axis must be equal
to `vec_size`).  It can then be broken along that axis, resulting in `vec_size` outputs.

`MuxComp` combines two or more inputs into a single output by stacking them along an axis.

## MuxComp and DemuxComp Options

These components have a single option, `vec_size`, which provides the number of inputs to be
combined into a single output (for `MuxComp`) or the number of outputs into which an input is
to be broken (for `DemuxComp`).  The default value of `vec_size` is 2.

## Adding Variables

A single `MuxComp` or `DemuxComp` can mux or demux multiple variables, so long as all variables
are compatible with the given `vec_size`.  Variables are added via the `add_var` method.

The axis along which the muxing/demuxing is to occur is given via the axis argument.
For DemuxComp, the specified axis index must be the index of one of the input dimensions (you cannot demux along axis 3 of a 2D input).
In addition, the axis on which the Demuxing is to be done must have length `vec_size`.

For MuxComp, the variables are joined along a new dimension, the index of which is given by axis.
The specified axis follows the convention used by the `numpy.stack` function.
Giving `axis = 0` will stack the inputs along the first axis (vertically).
Giving `axis = 1` will stack the inputs along the second axis (horizontally).
Giving `axis = -1` will stack the inputs along the last axis, and so is dependent on the shape of the inputs.
Due to the axis convention of `numpy.stack`, the axis index is only valid if it is less than or
equal to the number of dimensions in the inputs.
For example, 1D arrays can be stacked vertically (`axis = 0`) or horizontally (`axis = 1`), but not
depth-wise (`axis = 2`).

For DemuxComp, the name of the given variable is the **input**.  It is demuxed into variables whose
names are appended with `_n` where `n` is an integer from 0 through `vec_size`-1.
Conversely, for MuxComp, the given variable name is the output, and each input is appended with `_n`.

```{eval-rst}
    .. automethod:: openmdao.components.mux_comp.MuxComp.add_var
        :noindex:
```

```{eval-rst}
    .. automethod:: openmdao.components.demux_comp.DemuxComp.add_var
        :noindex:
```

## Example: Demuxing a 3-column matrix into constituent vectors

This example is contrived and could be achieved with a single vectorized component, but it serves
to give an example to the capabilities of the Demux component.  Given a position vector in the
Earth-centered, Earth-fixed (ECEF) frame (n x 3), extract the three (n x 1) columns from the matrix
and use the first two to compute the longitude at the given position vector.

In [ ]:
import numpy as np
import openmdao.api as om

# The number of elements to be demuxed
n = 3

# The size of each element to be demuxed
m = 100

p = om.Problem()

demux_comp = p.model.add_subsystem(name='demux', subsys=om.DemuxComp(vec_size=n),
                                 promotes_inputs=['pos_ecef'])

demux_comp.add_var('pos_ecef', shape=(m, n), axis=1, units='km')

p.model.add_subsystem(name='longitude_comp',
                      subsys=om.ExecComp('long = atan(y/x)',
                                         x={'val': np.ones(m), 'units': 'km'},
                                         y={'val': np.ones(m), 'units': 'km'},
                                         long={'val': np.ones(m), 'units': 'rad'}))

p.model.connect('demux.pos_ecef_0', 'longitude_comp.x')
p.model.connect('demux.pos_ecef_1', 'longitude_comp.y')

p.setup()

p.set_val('pos_ecef', 6378 * np.cos(np.linspace(0, 2*np.pi, m)), indices=om.slicer[:, 0])
p.set_val('pos_ecef', 6378 * np.sin(np.linspace(0, 2*np.pi, m)), indices=om.slicer[:, 1])
p.set_val('pos_ecef', 0.0, indices=om.slicer[:, 2])

p.run_model()

expected = np.arctan(p.get_val('pos_ecef', indices=om.slicer[:, 1]) / p.get_val('pos_ecef', indices=om.slicer[:, 0]))

print(p.get_val('longitude_comp.long'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('longitude_comp.long'), expected)

(om_slicer)=
## Example: Using `om.slicer` to reduce a 3-column matrix into constituent vectors

In [ ]:
# The number of elements to be demuxed
n = 3

arr_5x3 = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12],
    [13, 14, 15],
])

p = om.Problem()

p.model.add_subsystem('indep', om.IndepVarComp('x', arr_5x3, units='km'), promotes=['*'])
p.model.add_subsystem('indep2', om.IndepVarComp('y', arr_5x3, units='km'), promotes=['*'])

p.model.add_subsystem(name='longitude_comp',
                      subsys=om.ExecComp('long = atan(y/x)',
                                         x={'val': np.ones(n), 'units': 'km'},
                                         y={'val': np.ones(n), 'units': 'km'},
                                         long={'val': np.ones(n), 'units': 'rad'}))

# Use the src_indices arg in promotes to perform the demuxing
p.model.promotes('longitude_comp', inputs=['x'], src_indices=om.slicer[0, :])
p.model.promotes('longitude_comp', inputs=['y'], src_indices=om.slicer[1, :])


p.setup()

p.run_model()

print(p.get_val('longitude_comp.x'))
print(p.get_val('longitude_comp.y'))
print(p.get_val('longitude_comp.long'))

In [ ]:
assert(list(p.get_val('longitude_comp.x')) == [1, 2, 3])
assert(list(p.get_val('longitude_comp.y')) == [4, 5, 6])

## Example: Muxing 3 (n x 1) columns into a single (n x 3) matrix

In this example we start with three (n x 1) column vectors (`x`, `y`, and `z`) and
combine them into a single position vector `r` (n x 3).  This is achieved by stacking the vectors
along `axis = 1`.  Like the previous example, this is somewhat contrived but is intended to demonstrate
the capabilities of the MuxComp.

In [ ]:
# The number of elements to be muxed
n = 3

# The size of each element to be muxed
m = 100

p = om.Problem()

mux_comp = p.model.add_subsystem(name='mux', subsys=om.MuxComp(vec_size=n))

mux_comp.add_var('r', shape=(m,), axis=1, units='m')

p.model.add_subsystem(name='vec_mag_comp',
                      subsys=om.VectorMagnitudeComp(vec_size=m, length=n, in_name='r',
                                                    mag_name='r_mag', units='m'))

p.model.connect('mux.r', 'vec_mag_comp.r')

p.setup()

p.set_val('mux.r_0', 1 + np.random.rand(m))
p.set_val('mux.r_1', 1 + np.random.rand(m))
p.set_val('mux.r_2', 1 + np.random.rand(m))

p.run_model()

# Verify the results against numpy.dot in a for loop.
for i in range(n):
    r_i = [p.get_val('mux.r_0')[i], p.get_val('mux.r_1')[i], p.get_val('mux.r_2')[i]]
    expected_i = np.sqrt(np.dot(r_i, r_i))
    print(p.get_val('vec_mag_comp.r_mag')[i])

In [ ]:
# Verify the results against numpy.dot in a for loop.
for i in range(n):
    r_i = [p.get_val('mux.r_0')[i], p.get_val('mux.r_1')[i], p.get_val('mux.r_2')[i]]
    expected_i = np.sqrt(np.dot(r_i, r_i))
    assert_near_equal(p.get_val('vec_mag_comp.r_mag')[i], expected_i)